# Möbius‑Ladder Explorer (v3c) — *α‑Target Narrow Sweep*

Zeroes in on the sweet‑spot φ‑scale:

* **φ_scale grid**: 0.00360 → 0.00420 in **5 × 10⁻⁵** steps (13 points)  
* cycles fixed at **1** (extra cycles overshoot)  
* `seed_bits` = 64, 128  
* tiny exponents **k = 4, 5** (α⁻¹ / 10ᵏ) to test sensitivity

Prints any **< 1 %** hit (also flags < 0.1 %) and shows the 10 best rows.

In [4]:
import mpmath as mp, pandas as pd
mp.mp.dps = 120
ALPHA_INV = mp.mpf('137.035999084')
ALPHA = 1/ALPHA_INV
PHI = (1 + mp.sqrt(5))/2

def fib_word_bits(n):
    a,b='0','01'
    while len(b)<n:
        a,b=b,b+a
    return b[:n]

def fib_fraction(bits):
    return mp.fsum(int(b)*mp.mpf(2)**(-(k+1)) for k,b in enumerate(fib_word_bits(bits)))


In [5]:
def breath_shift(x, tiny, phi_scale):
    x = phi_scale*PHI / x
    x = x - tiny
    x = 1/x / 9
    x = 1/x
    x = x / 9
    return x

## Ultra‑narrow sweep

In [6]:
rows=[]
phi_vals = [0.0036 + i*0.00005 for i in range(0,13)]  # 0.00360 … 0.00420
for bits in (64,128):
    seed = fib_fraction(bits)
    for k_tiny in (4,5):
        tiny = ALPHA_INV / (10**k_tiny)
        for phi_scale in phi_vals:
            x = breath_shift(seed + tiny, tiny, phi_scale)
            rel = abs((x-ALPHA)/ALPHA)
            rows.append(dict(bits=bits, k=k_tiny, phi_scale=phi_scale, x=mp.nstr(x,25), rel_err=float(rel)))
            if rel < 0.01:
                print('Hit <1% | bits',bits,'k',k_tiny,'phi',phi_scale,'rel',rel)
                if rel < 0.001:
                    print('*** Hit <0.1% ***')
df = pd.DataFrame(rows)
display(df.sort_values('rel_err').head(10))


Hit <1% | bits 64 k 4 phi 0.0039499999999999995 rel 0.00407703280935272417835002778677536116947257344731630635503237352186609294222153098610165610252119449387517182501217813752
Hit <1% | bits 128 k 4 phi 0.0039499999999999995 rel 0.00407703280935272387985112952436319436190392861706143438489217317302339578030628221944777931007724104829216196907037772454


,bits,k,phi_scale,x,rel_err
7,64,4,0.00395,0.007327104115130185452272395,0.004077
33,128,4,0.00395,0.007327104115130185450094143,0.004077
32,128,4,0.00390,0.007060892671794361896129342,0.032404
6,64,4,0.00390,0.007060892671794361898280021,0.032404
8,64,4,0.00400,0.007593315558466013624297172,0.040558
34,128,4,0.00400,0.007593315558466013622091347,0.040558
31,128,4,0.00385,0.006794681228458538342164542,0.068884
5,64,4,0.00385,0.006794681228458538344287648,0.068884
35,128,4,0.00405,0.007859527001801837176056148,0.077038
9,64,4,0.00405,0.007859527001801837178289545,0.077038
